In [1]:
import ipywidgets as widgets
from fastai.vision import *
from fastbook import *
from fastai.vision.widgets import *
import librosa
from librosa import display
import numpy

c:\users\alexi\appdata\local\programs\python\python36\lib\site-packages\fastbook\__init__.py:22: UserWarning: Missing `graphviz` - please run `conda install fastbook`
  except ModuleNotFoundError: warn("Missing `graphviz` - please run `conda install fastbook`")


In [3]:
language = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(460),
    batch_tfms=aug_transforms(size=224)
)
path5='D:\\IA\\archive\\images'
dls=language.dataloaders(path5, bs=20, num_workers=0)

In [4]:
btn_string=widgets.Text(
    value='',
    placeholder='Type the path of the audio file',
    description='Path:',
    disabled=False
)
btn_upload = widgets.FileUpload()
btn_process = widgets.Button(description='create image')
btn_run = widgets.Button(description='Classify')
out_pl = widgets.Output()
lbl_pred = widgets.Label()

In [5]:
learn = cnn_learner(dls, resnet34, metrics=(accuracy, error_rate))
learn.load('C:\\Users\\alexi\\Downloads\\models\\stage-3')

In [6]:
def save_spectrogram(audio_fname):
    y, sr =librosa.load(audio_fname, sr=None)
    S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
    log_S = librosa.power_to_db(S, ref=np.max)
    librosa.display.specshow(log_S, sr=sr, x_axis='time', y_axis='mel')
    fig1 = plt.gcf()
    fig1.savefig(audio_fname.replace('flac','png'), dpi=100)

In [7]:
def on_click_process(change):
    save_spectrogram(btn_string.value)
    
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    #with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
    
btn_process.on_click(on_click_process)
btn_run.on_click(on_click_classify)

In [9]:
VBox([widgets.Label('Select your audio!'), 
      btn_string,btn_process,widgets.Label('Select the image created'),btn_upload, btn_run, out_pl, lbl_pred])